In [1]:
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils

import torch
from datasets import load_dataset

from IPython.display import HTML, display

torch.set_grad_enabled(False)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = HookedTransformer.from_pretrained('gpt2-xl')

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

In [ ]:
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
from sae_lens.training.sparse_autoencoder import SparseAutoencoder
from sae_lens.training.config import LanguageModelSAERunnerConfig

hook_point = "blocks.20.hook_resid_pre"
bs = 64

cf = {
  "model_name": "gpt2-xl",
  "hook_point": "blocks.20.hook_resid_pre",
  "hook_point_layer": 20,
  "hook_point_head_index": None,
  "dataset_path": "Skylion007/openwebtext",
  "is_dataset_tokenized": False,
  "context_size": 128,
  "use_cached_activations": False,
  "cached_activations_path": "activations/Skylion007_openwebtext/gpt2-small/blocks.1.hook_resid_pre",
  "d_in": 1600,
  "n_batches_in_buffer": bs,
  "total_training_tokens": 300000000,
  "store_batch_size": bs,
  "device": device,
  "seed": 42,
  "dtype": "torch.float16",
  "b_dec_init_method": "geometric_median",
  "expansion_factor": 32,
  "from_pretrained_path": None,
  "l1_coefficient": 0.00008,
  "lr": 0.0004,
  "lr_scheduler_name": None,
  "lr_warm_up_steps": 5000,
  "train_batch_size": 4096,
  "use_ghost_grads": False,
  "feature_sampling_window": 1000,
  "feature_sampling_method": None,
  "resample_batches": 1028,
  "feature_reinit_scale": 0.2,
  "dead_feature_window": 5000,
  "dead_feature_estimation_method": "no_fire",
  "dead_feature_threshold": 1e-8,
  "log_to_wandb": True,
  "wandb_project": "mats_sae_training_gpt2_small_resid_pre_5",
  "wandb_entity": None,
  "wandb_log_frequency": 100,
  "n_checkpoints": 10,
  "checkpoint_path": "checkpoints/mm179kd2",
  "d_sae": 1600*32,
  "tokens_per_buffer": 128,
  "run_name": "24576-L1-8e-05-LR-0.0004-Tokens-3.000e+08"
}
config = cf
var_names = LanguageModelSAERunnerConfig.__init__.__code__.co_varnames
config = {k: v for k, v in config.items() if k in var_names}
cfg = LanguageModelSAERunnerConfig(
    **config
)
sparse_autoencoder = SparseAutoencoder(cfg)
sparse_autoencoder.to(device)
sparse_autoencoder.cfg.device = device

layer = cfg.hook_point_layer


In [ ]:
from safetensors import safe_open
tensors = {}
with safe_open("gpt2-20.safetensors", framework="pt") as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k).cuda()
        # if k == "b_enc":
        #     tensors[k] -= 0.18
tensors["b_enc"] += tensors["b_dec"] @ tensors["W_enc"]
sparse_autoencoder.load_state_dict(tensors, strict=False)
sparse_autoencoder.to(device)
sparse_autoencoder.cfg.device = device

In [ ]:
def top_acts_at_pos(text, pos=-1, silent=True, prepend_bos=True, n_top=10):
    logits, cache = model.run_with_cache(text, prepend_bos=prepend_bos)
    if pos is None:
        hidden_state = cache[hook_point][0, :, :]
    else:
        hidden_state = cache[hook_point][0, pos, :].unsqueeze(0)
    feature_acts = sparse_autoencoder(hidden_state).feature_acts
    feature_acts = feature_acts.mean(dim=0)
    top_v, top_i = torch.topk(feature_acts, n_top)
    return top_v, top_i

top_acts_at_pos("Anger something something", pos=-1)


In [ ]:
top_acts_at_pos("Anger", pos=-1) # [126, 20811, 4524 ...]

In [ ]:
# # data = load_dataset("NeelNanda/c4-code-20k", split="train")
# data = load_dataset("NeelNanda/pile-10k", split="train")

# tokenized_data = tutils.tokenize_and_concatenate(data, model.tokenizer, max_length=128)
# tokenized_data = tokenized_data.shuffle(42)
# all_tokens = tokenized_data["tokens"]

In [ ]:
from tqdm.auto import tqdm

from sae_lens.training.session_loader import LMSparseAutoencoderSessionloader
import transformer_lens.loading_from_pretrained as tllfp
loader = LMSparseAutoencoderSessionloader(sparse_autoencoder.cfg)
_, _, activation_store = loader.load_sae_training_group_session()


def get_tokens(
    activation_store,
    n_batches_to_sample_from: int = 2**13,
    n_prompts_to_select: int = 4096 * 6,
):
    all_tokens_list = []
    pbar = tqdm(range(n_batches_to_sample_from))
    for _ in pbar:
        batch_tokens = activation_store.get_batch_tokens()
        batch_tokens = batch_tokens[torch.randperm(batch_tokens.shape[0])][
            : batch_tokens.shape[0]
        ]
        all_tokens_list.append(batch_tokens)

    all_tokens = torch.cat(all_tokens_list, dim=0)
    all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
    return all_tokens[:n_prompts_to_select]


all_tokens = get_tokens(activation_store)  # should take a few minutes

In [ ]:
all_tokens.shape

In [ ]:
# okay so 126 is the anger feature right? Wrong!
# 126 activates on most text
selected_feature = 126
activation_count = 0
total = 0
for i in range(10):
    logits, cache = model.run_with_cache(all_tokens[i])
    hidden_state = cache[hook_point][0]
    feature_acts = sparse_autoencoder(hidden_state).feature_acts # shape [128, n_features]
    selected_acts = feature_acts[:, selected_feature]

    activation_count += (selected_acts > 0).sum().item()
    total += selected_acts.shape[0]

print(activation_count/total)
# this feature activates on 99% of all tokens!!!

In [ ]:
from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData

test_feature_idx_gpt = [126, 20811, 409]

feature_vis_config_gpt = SaeVisConfig(
    hook_point=hook_point,
    features=test_feature_idx_gpt,
    batch_size=bs,
    minibatch_size_tokens=128,
    verbose=True,
)

with torch.inference_mode():
    sae_vis_data_gpt = SaeVisData.create(
        encoder=sparse_autoencoder,
        model=model,
        tokens=all_tokens,  # type: ignore
        cfg=feature_vis_config_gpt,
    )

In [ ]:
import os
from IPython.display import FileLink

vis_dir = "feature_vis"
if not os.path.exists(vis_dir):
    os.makedirs(vis_dir)

for idx, feature in enumerate(test_feature_idx_gpt):
    if sae_vis_data_gpt.feature_stats.max[idx] == 0:
        continue
    filename = os.path.join(vis_dir, f"{feature}_feature_vis.html")
    sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
    display(FileLink(filename))

In [ ]:
top_acts_at_pos(" any text you like!", pos=-1)

In [ ]:
# also, another crazy thing about this SAE:
# the 0th token causes some features to activate with crazy high magnitude
top_acts_at_pos(" hello", pos=0)

In [ ]:
# same thing with and without BOS
top_acts_at_pos(" hello", pos=0, prepend_bos=False)

In [ ]:
# my takeaway is this is a bad SAE.

In [ ]:
# checking that you can indeed find a reasonable "Angry" feature.
# get features for angry, remove features which also activate on "Calm"

angry_vals, angry_ids = top_acts_at_pos("Anger", pos=-1, n_top=100)
calm_vals, calm_ids = top_acts_at_pos("Calm", pos=None, n_top=200)

angry_vals = [t.item() for t in angry_vals]
angry_ids = [t.item() for t in angry_ids]
calm_vals = [t.item() for t in calm_vals]
calm_ids = [t.item() for t in calm_ids]

angry = zip(angry_vals, angry_ids)
calm = zip(calm_vals, calm_ids)

# remove zero-valued calm ids
calm = [(v, i) for v, i in calm if v > 0]
calm_set = set([i for v, i in calm])

# remove calm ids from angry
angry = [(v, i) for v, i in angry if i not in calm_set]

In [ ]:
angry[:10]

In [ ]:
test_feature_idx_gpt = [20811, 4364, 33085]

feature_vis_config_gpt = SaeVisConfig(
    hook_point=hook_point,
    features=test_feature_idx_gpt,
    batch_size=bs,
    minibatch_size_tokens=128,
    verbose=False,
)

with torch.inference_mode():
    sae_vis_data_gpt = SaeVisData.create(
        encoder=sparse_autoencoder,
        model=model,
        tokens=all_tokens,  # type: ignore
        cfg=feature_vis_config_gpt,
    )

for idx, feature in enumerate(test_feature_idx_gpt):
    if sae_vis_data_gpt.feature_stats.max[idx] == 0:
        continue
    filename = os.path.join(vis_dir, f"{feature}_feature_vis.html")
    sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
    display(FileLink(filename))

In [ ]:
# 20811 # angry/sad/irritated feature
# 4364 activates on words that end in "er" or "ner"
# 33085 violence/aggression feature